# Review Instructions

Please review the MSv4 `field_and_source_xds` schema and the XRADIO interface (`ps['MSv4_name'].VISIBILITY.field_and_source_xds`). Note that the PS (processing set) interface or the main_xds should not be reviewed.

The `field_and_source_xds` schema specification: https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1658760192

## Preparatory Material
Go over Xarray nomenclature and selection syntax:
- https://docs.xarray.dev/en/latest/user-guide/terminology.html
- https://docs.xarray.dev/en/latest/user-guide/indexing.html

MSv2 and CASA documentation:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf
- Ephemeris Data in CASA: https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data

## `field_and_source_xds` Schema
The FIELD, SOURCE, and EPHEMERIS tables in the MSv2 contain closely related information:
- **FIELD**: Field position for a source.
- **SOURCE**: Information about the source being observed (position, proper motion, etc.)
- **EPHEMERIS**: Ephemerides of the source.

These can be combined into a single dataset for MSv4 because it consists of a single field and consequently a single source[^1].

### Use Cases
The use cases considered during the design of the schema were:
- Single field observation (type=standard).
- Mosaic observation (type=standard).
- Ephemeris observation (type=ephemeris).
- Mosaic Ephemeris observation (type=ephemeris).

To satisfy these use cases, two types of `field_and_source_xds` were created: standard and ephemeris. The main difference is that the ephemeris type has a `FIELD_PHASE_OFFSET` data variable that is relative to the `SOURCE_POSITION/SOURCE_DIRECTION` data variable (contains the ephemerides and has a time axis), while the standard type has `FIELD_PHASE/DELAY/REFERENCE_CENTERS` and `SOURCE_POSITION` (has no time axis). The `SOURCE_POSITION/DIRECTION` is kept separate from the `FIELD_PHASE_OFFSET/CENTER` so that the intent `OBSERVE_TARGET#OFF_SOURCE` is supported and the ephemeris can be easily changed.

## Key Questions to Answer
### Schema Questions
- 1.1) Are there missing use cases?
- 1.2) Is all the information present needed for offline processing?
  - 1.2.1) Are there data variables we need to add (for example, the JPL Horizons data has additional information such as the North pole position angle, etc., see [CASA Ephemeris Data](https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data))?
  - 1.2.2) In some CASA ephemeris tables, there are table keywords such as `VS_CREATE`, `VS_DATE`, `VS_TYPE`, `VS_VERSION`, `MJD0`, `dMJD`, `earliest`, `latest`, `radii`, `meanradm`, `orb_per`, `rot_per`. Do we need any of these?
- 1.3) Is there a use case where the `FIELD_PHASE_CENTER` and `FIELD_DELAY_CENTER` would differ (i.e., do we need to store both)?
- 1.4) For interferometer observations, do we need to store the `FIELD_REFERENCE_CENTER` or can this be omitted (will still be present for Single dish)?
- 1.5) The ephemeris data is recorded in degrees, AU, and MJD. Should these be converted to radians, meters, and time (Unix)? Note that each data variable has measurement information attached to it. For example:
```Python
  frame: ICRS
  type: sky_coord
  units: ['deg', 'deg', 'AU']
```

- 1.6) For ephemeris observations, should we add the SOURCE_PROPER_MOTION if available?
- 1.7) Is the name `field_and_source_xds` sufficiently descriptive?
- 1.8) Should we also add the DOPPLER table information to the schema (if so, any idea where we can get an MSv2 with a DOPPLER table)?
- 1.9) Any naming suggestions or data layout?
- 1.10) Are the data variable descriptions in the schema spreadsheet correct?
- 1.11) What measures (https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1504318014) should we attach to each of the following data variables

  - NORTH_POLE_POSITION_ANGLE (quantity?)
  - NORTH_POLE_ANGULAR_DISTANCE (quantity?)
  - SUB_OBSERVER_DIRECTION (earth_location?)
  - SUB_SOLAR_POSITION (earth_location?)
  - HELIOCENTRIC_RADIAL_VELOCITY (quantity?)
  - OBSERVER_PHASE_ANGLE (quantity?)
- 1.12) Can NORTH_POLE_POSITION_ANGLE and NORTH_POLE_ANGULAR_DISTANCE be combined into a single data variable?

### XRADIO
2.1) After reviewing the XARRAY documentation and the descriptions of the data variables in the `field_and_source_xds` schema, do you find the XARRAY interface intuitive and easy to use?



[^1]: This is inhereted from MSv2 that only allows a single source per field [https://casacore.github.io/casacore-notes/229.pdf, p35], though a source can appear in more than one field.


# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 162-create-combined-field-source-and-ephemeris-dataset
cd xradio
pip install -e .
```

# ALMA Example

An ephemeris mosaic observation of the sun.

ALMA archive file downloaded: https://almascience.nrao.edu/dataPortal/2022.A.00001.S_uid___A002_X1003af4_X75a3.asdm.sdm.tar 

- Project: 2022.A.00001.S
- Member ous id (MOUS): uid://A001/X3571/X130
- Group ous id (GOUS): uid://A001/X3571/X131

CASA commands used to create the dataset:
```python
importasdm(asdm='uid___A002_X1003af4_X75a3.asdm.sdm',vis='uid___A002_X1003af4_X75a3.ms',asis='Ephemeris Antenna Station Receiver Source CalAtmosphere CalWVR',bdfflags=True,with_pointing_correction=True,convert_ephem2geo=True)

importasdm(asdm='22A-347.sb41163051.eb41573599.59642.62832791667.asdm',vis='22A-347.sb41163051.eb41573599.59642.62832791667.ms',asis='Ephemeris Antenna Station Receiver Source',with_pointing_correction=True,convert_ephem2geo=True)

mstransform(vis='ALMA_uid___A002_X1003af4_X75a3.split.ms',outputvis='ALMA_uid___A002_X1003af4_X75a3.split.avg.ms',createmms=False,timeaverage=True,timebin='2s',timespan='scan',reindex=True)

import numpy as np
for subtable in ['FLAG_CMD', 'POINTING', 'CALDEVICE', 'ASDM_CALATMOSPHERE']:
    tb.open('ALMA_uid___A002_X1003af4_X75a3.split.avg.ms::'+subtable,nomodify=False)
    tb.removerows(np.arange(tb.nrows())) 
    tb.flush()
    tb.done()
```


## Download Data

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="ALMA_uid___A002_X1003af4_X75a3.split.avg.ms")

[2024-08-05 10:18:24,905]  WARNING  graphviper:  File exists: /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox 
[2024-08-05 10:18:24,912]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List                                
 ───────────────────────────────────────────── 
  ALMA_uid___A002_X1003af4_X75a3.split.avg.ms

[2024-08-05 10:18:25,799]     INFO  graphviper:  File exists: ALMA_uid___A002_X1003af4_X75a3.split.avg.ms 


# Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [2]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-08-05 10:18:26,032]     INFO  graphviper:  Checking parameter values for client.local_client 
[2024-08-05 10:18:26,032]     INFO  graphviper:  Module path: /Users/jsteeb/Dropbox/graphviper/ 
[2024-08-05 10:18:26,035]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 


/Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54954 instead
  warnings.warn(


[2024-08-05 10:18:26,829]     INFO      client:  Created client <MenrvaClient: 'tcp://127.0.0.1:54955' processes=4 threads=4, memory=14.90 GiB> 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:54954/status,
Dashboard: http://127.0.0.1:54954/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54955,Workers: 4
Dashboard: http://127.0.0.1:54954/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:54966,Total threads: 1
Dashboard: http://127.0.0.1:54970/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:54958,


# Convert dataset

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
import os

in_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.ms"
out_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr"
os.system("rm -rf "+out_file)

#partition_scheme=[] #must be ephemeris_interpolate=True
partition_scheme=['FIELD_ID'] #can be ephemeris_interpolate=True/False

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    parallel=True,
    overwrite=True,
    ephemeris_interpolate=True,
    partition_scheme=partition_scheme
)



[2024-08-05 10:18:28,980]     INFO      client:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBS_MODE', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-05 10:18:29,610]     INFO      client:  Number of partitions: 96 
[2024-08-05 10:18:29,610]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [0], FIELD [0], SCAN [7] 
[2024-08-05 10:18:29,611]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [16], FIELD [0], SCAN [7] 
[2024-08-05 10:18:29,612]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [17], FIELD [0], SCAN [7] 
[2024-08-05 10:18:29,612]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [19], FIELD [2], SCAN [6] 
[2024-08-05 10:18:29,612]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [20 99], FIELD [0], SCAN [8] 
[2024-08-05 10:18:29,613]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [21 48 75], FIELD [3], SCAN [8] 
[2024-08-05 10:18:29,613]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [22 49 76], FIELD [4],

# Inspect Processing Set

In [4]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.summary()

,name,obs_mode,shape,polarization,spw_name,field_name,source_name,field_coords,start_frequency,end_frequency
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_52,OBSERVE_TARGET#ON_SOURCE,"(9, 51, 4, 1)",[XX],WVR#NOMINAL,[Sun_10_18],[Sun_10_0],Ephemeris,1.845500e+11,1.905500e+11
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_55,OBSERVE_TARGET#ON_SOURCE,"(9, 51, 4, 1)",[XX],WVR#NOMINAL,[Sun_10_21],[Sun_10_0],Ephemeris,1.845500e+11,1.905500e+11
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_63,OBSERVE_TARGET#ON_SOURCE,"(6, 51, 4, 1)",[XX],WVR#NOMINAL,[Sun_10_29],[Sun_10_0],Ephemeris,1.845500e+11,1.905500e+11
3,ALMA_uid___A002_X1003af4_X75a3.split.avg_90,OBSERVE_TARGET#ON_SOURCE,"(9, 1326, 7, 2)","[XX, YY]",X767114449#ALMA_RB_06#BB_1#SW-01#FULL_RES,[Sun_10_24],[Sun_10_0],Ephemeris,2.299609e+11,2.300547e+11
4,ALMA_uid___A002_X1003af4_X75a3.split.avg_64,"CALIBRATE_ATMOSPHERE#OFF_SOURCE,CALIBRATE_WVR#...","(2, 1326, 7, 2)","[XX, YY]",X767114449#ALMA_RB_06#BB_1#SW-01#FULL_RES,[Sun_10_0],[Sun_10_0],Ephemeris,2.299609e+11,2.300547e+11
5,ALMA_uid___A002_X1003af4_X75a3.split.avg_30,OBSERVE_TARGET#ON_SOURCE,"(8, 51, 1, 2)","[XX, YY]",X767114449#ALMA_RB_06#BB_4#SQLD,[Sun_10_28],[Sun_10_0],Ephemeris,2.480000e+11,2.480000e+11
6,ALMA_uid___A002_X1003af4_X75a3.split.avg_37,OBSERVE_TARGET#ON_SOURCE,"(9, 51, 4, 1)",[XX],WVR#NOMINAL,[Sun_10_3],[Sun_10_0],Ephemeris,1.845500e+11,1.905500e+11
7,ALMA_uid___A002_X1003af4_X75a3.split.avg_39,OBSERVE_TARGET#ON_SOURCE,"(9, 51, 4, 1)",[XX],WVR#NOMINAL,[Sun_10_5],[Sun_10_0],Ephemeris,1.845500e+11,1.905500e+11
8,ALMA_uid___A002_X1003af4_X75a3.split.avg_65,"CALIBRATE_ATMOSPHERE#AMBIENT,CALIBRATE_WVR#AMB...","(2, 1326, 7, 2)","[XX, YY]",X767114449#ALMA_RB_06#BB_1#SW-01#FULL_RES,[Sun_10_0],[Sun_10_0],Ephemeris,2.299609e+11,2.300547e+11
9,ALMA_uid___A002_X1003af4_X75a3.split.avg_91,OBSERVE_TARGET#ON_SOURCE,"(9, 1326, 7, 2)","[XX, YY]",X767114449#ALMA_RB_06#BB_1#SW-01#FULL_RES,[Sun_10_25],[Sun_10_0],Ephemeris,2.299609e+11,2.300547e+11


In [5]:
partition_scheme = []
if partition_scheme:
    msv4_name_ephemeris = "ALMA_uid___A002_X1003af4_X75a3.split.avg_67"
    msv4_name = "ALMA_uid___A002_X1003af4_X75a3.split.avg_52"
else:
    msv4_name_ephemeris = "ALMA_uid___A002_X1003af4_X75a3.split.avg_17"
    msv4_name = "ALMA_uid___A002_X1003af4_X75a3.split.avg_3"
    
msv4_name

'ALMA_uid___A002_X1003af4_X75a3.split.avg_3'

In [6]:
ps[msv4_name].partition_info

{'field_name': ['J1408-0752_2'],
 'obs_mode': 'CALIBRATE_PHASE#ON_SOURCE,CALIBRATE_WVR#ON_SOURCE',
 'polarization_setup': ['XX', 'YY'],
 'source_name': ['J1408-0752_2'],
 'spectral_window_name': 'X767114449#ALMA_RB_06#BB_4#SQLD',
 'taql': 'WHERE (DATA_DESC_ID IN [0]) AND(OBSERVATION_ID IN [0]) AND(STATE_ID IN [19]) AND(FIELD_ID IN [2]) AND(SCAN_NUMBER IN [6]) AND(STATE_ID IN [19]) '}

In [7]:
ps[msv4_name]

<xarray.Dataset> Size: 109kB
Dimensions:                     (time: 31, baseline_id: 51, frequency: 1,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_name      (baseline_id) <U9 2kB dask.array<chunksize=(51,), meta=np.ndarray>
    baseline_antenna2_name      (baseline_id) <U9 2kB dask.array<chunksize=(51,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 408B 0 1 2 3 ... 47 48 49 50
  * frequency                   (frequency) float64 8B 2.48e+11
  * polarization                (polarization) <U2 16B 'XX' 'YY'
  * time                        (time) float64 248B 1.667e+09 ... 1.667e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 13kB dask.array<chunksize=(31, 51), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 3kB dask.array<chunksize=(31, 51, 1, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 13kB dask.array<chunksize=(31, 51), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 38kB dask.array<chunksize=(31, 51, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 25kB dask.array<chunksize=(31, 51, 1, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 13kB dask.array<chunksize=(31, 51, 1, 2), meta=np.ndarray>
Attributes:
    data_groups:     {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'V...
    partition_info:  {'field_name': ['J1408-0752_2'], 'obs_mode': 'CALIBRATE_...
    pointing_xds:    <xarray.Dataset> Size: 0B\nDimensions:  ()\nData variabl...
    weather_xds:     <xarray.Dataset> Size: 138kB\nDimensions:         (stati...
    antenna_xds:     <xarray.Dataset> Size: 6kB\nDimensions:                (...

# Inspect field_and_source_xds: Standard Use case (non-ephemeris)

In [8]:
standard_field_and_source_xds = ps[
    msv4_name
].VISIBILITY.field_and_source_xds.load()  # Load the data into memory
standard_field_and_source_xds

<xarray.Dataset> Size: 152B
Dimensions:             (sky_dir_label: 2)
Coordinates:
    field_name          <U12 48B 'J1408-0752_2'
  * sky_dir_label       (sky_dir_label) <U3 24B 'ra' 'dec'
    source_name         <U12 48B 'J1408-0752_2'
Data variables:
    FIELD_PHASE_CENTER  (sky_dir_label) float64 16B -2.579 -0.1374
    SOURCE_DIRECTION    (sky_dir_label) float64 16B -2.579 -0.1374
Attributes:
    is_ephemeris:  False

In [9]:
standard_field_and_source_xds  # How to access field_and_source_xds.

<xarray.Dataset> Size: 152B
Dimensions:             (sky_dir_label: 2)
Coordinates:
    field_name          <U12 48B 'J1408-0752_2'
  * sky_dir_label       (sky_dir_label) <U3 24B 'ra' 'dec'
    source_name         <U12 48B 'J1408-0752_2'
Data variables:
    FIELD_PHASE_CENTER  (sky_dir_label) float64 16B -2.579 -0.1374
    SOURCE_DIRECTION    (sky_dir_label) float64 16B -2.579 -0.1374
Attributes:
    is_ephemeris:  False

In [10]:
standard_field_and_source_xds.FIELD_PHASE_CENTER  # How to access field_and_source_xds datavariables. standard_field_and_source_xds['FIELD_PHASE_CENTER'] can also be used.

<xarray.DataArray 'FIELD_PHASE_CENTER' (sky_dir_label: 2)> Size: 16B
array([-2.57897986, -0.13742852])
Coordinates:
    field_name     <U12 48B 'J1408-0752_2'
  * sky_dir_label  (sky_dir_label) <U3 24B 'ra' 'dec'
    source_name    <U12 48B 'J1408-0752_2'
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad']

In [11]:
standard_field_and_source_xds.FIELD_PHASE_CENTER.attrs  # How to access field_and_source_xds datavariables measures information stored in the attributes.

{'frame': 'ICRS', 'type': 'sky_coord', 'units': ['rad', 'rad']}

# Inspect field_and_source_xds: Ephemeris Use case (Mosaic) with line

In [12]:
ephemeris_field_and_source_xds = ps[
    msv4_name_ephemeris
].VISIBILITY.field_and_source_xds.load()  # Load the data into memory
ephemeris_field_and_source_xds

<xarray.Dataset> Size: 2kB
Dimensions:                       (time: 12, sky_pos_label: 3,
                                   ellipsoid_pos_label: 3, sky_dir_label: 2)
Coordinates:
  * ellipsoid_pos_label           (ellipsoid_pos_label) <U4 48B 'lon' ... 'dist'
    field_name                    <U9 36B 'Sun_10_15'
  * sky_dir_label                 (sky_dir_label) <U3 24B 'ra' 'dec'
  * sky_pos_label                 (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
    source_name                   <U8 32B 'Sun_10_0'
Dimensions without coordinates: time
Data variables:
    FIELD_PHASE_CENTER            (time, sky_pos_label) float64 288B -2.624 ....
    HELIOCENTRIC_RADIAL_VELOCITY  (time) float64 96B 0.0 0.0 0.0 ... 0.0 0.0 0.0
    NORTH_POLE_ANGULAR_DISTANCE   (time) float64 96B 543.8 543.8 ... 548.6 548.6
    NORTH_POLE_POSITION_ANGLE     (time) float64 96B 14.53 14.53 ... 14.65 14.65
    OBSERVATION_POSITION          (ellipsoid_pos_label) float64 24B 0.0 0.0 0.0
    OBSERVER_PHASE_ANGLE          (time) float64 96B 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SOURCE_POSITION               (time, sky_pos_label) float64 288B 3.659 .....
    SOURCE_RADIAL_VELOCITY        (time) float64 96B -204.8 -204.8 ... -202.4
    SUB_OBSERVER_POSITION         (time, ellipsoid_pos_label) float64 288B 3....
Attributes:
    is_ephemeris:  True

In [13]:
ephemeris_field_and_source_xds.FIELD_PHASE_CENTER

<xarray.DataArray 'FIELD_PHASE_CENTER' (time: 12, sky_pos_label: 3)> Size: 288B
array([[-2.62415778e+00, -2.11208761e-01,  1.48765931e+11],
       [-2.62415739e+00, -2.11208901e-01,  1.48765930e+11],
       [-2.62415700e+00, -2.11209040e-01,  1.48765929e+11],
       [-2.62415681e+00, -2.11209111e-01,  1.48765929e+11],
       [-2.62411646e+00, -2.11223659e-01,  1.48765830e+11],
       [-2.62411607e+00, -2.11223798e-01,  1.48765829e+11],
       [-2.62411568e+00, -2.11223937e-01,  1.48765828e+11],
       [-2.62411549e+00, -2.11224009e-01,  1.48765828e+11],
       [-2.62407514e+00, -2.11238556e-01,  1.48765730e+11],
       [-2.62407475e+00, -2.11238695e-01,  1.48765729e+11],
       [-2.62407436e+00, -2.11238835e-01,  1.48765728e+11],
       [-2.62407417e+00, -2.11238906e-01,  1.48765727e+11]])
Coordinates:
    field_name     <U9 36B 'Sun_10_15'
  * sky_pos_label  (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
    source_name    <U8 32B 'Sun_10_0'
Dimensions without coordinates: time
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad', 'm']

In [14]:
ephemeris_field_and_source_xds.SOURCE_POSITION

<xarray.DataArray 'SOURCE_POSITION' (time: 12, sky_pos_label: 3)> Size: 288B
array([[ 3.65894029e+00, -2.11183577e-01,  1.48765931e+11],
       [ 3.65894068e+00, -2.11183717e-01,  1.48765930e+11],
       [ 3.65894106e+00, -2.11183856e-01,  1.48765929e+11],
       [ 3.65894126e+00, -2.11183927e-01,  1.48765929e+11],
       [ 3.65898161e+00, -2.11198475e-01,  1.48765830e+11],
       [ 3.65898200e+00, -2.11198614e-01,  1.48765829e+11],
       [ 3.65898238e+00, -2.11198753e-01,  1.48765828e+11],
       [ 3.65898258e+00, -2.11198825e-01,  1.48765828e+11],
       [ 3.65902293e+00, -2.11213372e-01,  1.48765730e+11],
       [ 3.65902332e+00, -2.11213511e-01,  1.48765729e+11],
       [ 3.65902370e+00, -2.11213651e-01,  1.48765728e+11],
       [ 3.65902390e+00, -2.11213722e-01,  1.48765727e+11]])
Coordinates:
    field_name     <U9 36B 'Sun_10_15'
  * sky_pos_label  (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
    source_name    <U8 32B 'Sun_10_0'
Dimensions without coordinates: time
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad', 'm']

In [15]:
ephemeris_field_and_source_xds.SOURCE_POSITION.sel(sky_pos_label="dec")

<xarray.DataArray 'SOURCE_POSITION' (time: 12)> Size: 96B
array([-0.21118358, -0.21118372, -0.21118386, -0.21118393, -0.21119847,
       -0.21119861, -0.21119875, -0.21119882, -0.21121337, -0.21121351,
       -0.21121365, -0.21121372])
Coordinates:
    field_name     <U9 36B 'Sun_10_15'
    sky_pos_label  <U4 16B 'dec'
    source_name    <U8 32B 'Sun_10_0'
Dimensions without coordinates: time
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad', 'm']